# Settings

## Installations

In [0]:
%pip install playwright nest_asyncio
dbutils.library.restartPython()

  Obtaining dependency information for playwright from https://files.pythonhosted.org/packages/56/61/3a803cb5ae0321715bfd5247ea871d25b32c8f372aeb70550a90c5f586df/playwright-1.57.0-py3-none-manylinux1_x86_64.whl.metadata
  Using cached playwright-1.57.0-py3-none-manylinux1_x86_64.whl.metadata (3.5 kB)
  Obtaining dependency information for pyee<14,>=13 from https://files.pythonhosted.org/packages/9b/4d/b9add7c84060d4c1906abe9a7e5359f2a60f7a9a4f67268b2766673427d8/pyee-13.0.0-py3-none-any.whl.metadata
  Using cached pyee-13.0.0-py3-none-any.whl.metadata (2.9 kB)
  Obtaining dependency information for greenlet<4.0.0,>=3.1.1 from https://files.pythonhosted.org/packages/5c/ae/8d472e1f5ac5efe55c563f3eabb38c98a44b832602e12910750a7c025802/greenlet-3.3.1-cp311-cp311-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata
  Using cached greenlet-3.3.1-cp311-cp311-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (3.7 kB)
Using cached playwright-1.57.0-py3-none-manylinux1_x86_64.whl (46.0 

## Imports

In [0]:
import subprocess
import sys
import re
import asyncio
from datetime import datetime
from io import StringIO
import logging
from typing import Optional
import requests
import pandas as pd
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
import nest_asyncio

nest_asyncio.apply()

## System Setup

In [0]:
# Installs browser binaries (Chromium) and necessary OS dependencies 

def install_playwright_system_deps():
    print("Starting Playwright system setup...")
    
    try:
        # Install the browser binaries
        print("Installing browsers...")
        subprocess.run([sys.executable, "-m", "playwright", "install"], check=True)
        
        # Install OS-level dependencies (replaces manual apt-get commands)
        print("Installing OS dependencies (install-deps)...")
        subprocess.run([sys.executable, "-m", "playwright", "install-deps"], check=True)
        
        print("Successfully installed Playwright and dependencies.")
        
    except subprocess.CalledProcessError as e:
        print(f"Error during installation: {e}")
        raise

install_playwright_system_deps()

Starting Playwright system setup...
Installing browsers...
Installing OS dependencies (install-deps)...
Installing dependencies...
Hit:1 https://repos.azul.com/zulu/deb stable InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists...
Reading package lists...

W: https://repos.azul.com/zulu/deb/dists/stable/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.



Building dependency tree...
Reading state information...
fonts-freefont-ttf is already the newest version (20120503-10build1).
fonts-liberation is already the newest version (1:1.07.4-11).
libasound2 is already the newest version (1.2.6.1-1ubuntu1).
libatk-bridge2.0-0 is already the newest version (2.38.0-3).
libatk1.0-0 is already the newest version (2.36.0-3build1).
libatspi2.0-0 is already the newest version (2.44.0-3).
libcairo-gobject2 is already the newest version (1.16.0-5ubuntu2).
libcairo2 is already the newest version (1.16.0-5ubuntu2).
libdbus-glib-1-2 is already the newest version (0.112-2build1).
libegl1 is already the newest version (1.4.0-1).
libenchant-2-2 is already the newest version (2.3.2-1ubuntu2).
libepoxy0 is already the newest version (1.5.10-1).
libevdev2 is already the newest version (1.12.1+dfsg-1).
libevent-2.1-7 is already the newest version (2.1.12-stable-1build3).
libfontconfig1 is already the newest version (2.13.1-4.2ubuntu5).
libgles2 is already the n

## Configuration & Constants

In [0]:
WC_SCHEDULE_URL = "https://www.mlssoccer.com/news/fifa-world-cup-2026-schedule-every-game-by-city-stadium"
LATLONG_URL = "https://www.latlong.net/location/2026-fifa-world-cup-locations-2252"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}

DELTA_TABLE_NAME = "world_cup_anastasia"

COUNTRIES = {"United States", "Canada", "Mexico"}
STADIUM_SYNONYMS = {
    "Estadio Azteca": "Estadio Banorte", 
    "Mexico City Stadium": "Estadio Banorte",
    "Hard Rock Stadium": "Hard Rock Stadium",
    "Gillette Stadium": "Gillette Stadium",
    "MetLife Stadium": "MetLife Stadium",
    "Mercedes-Benz Stadium": "Mercedes-Benz Stadium"
}

MONTH_RE = re.compile(r"^(?:[\u2022•\*\-\u2013\u2014]*\s*)?(June|July)\s+(\d{1,2})(?::)?\s*(.*)$")
STADIUM_HDR_RE = re.compile(r"^(?P<city>.+?)\s*[\-\u2013\u2014]\s*(?P<stadium>.+)$")
PACKED_LATLON_RE = re.compile(r"^\s*([+\-]?\d+(?:\.\d+)?)([+\-]\d+(?:\.\d+)?)\s*$")

# Core Data Pipeline: Scraping & Parsing

## Network Utility Functions

In [0]:
# Setup basic logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def get_html_static(url: str) -> str:
    """
    Fetches HTML content using standard HTTP requests.
    Best for static pages like LatLong.net.
    """
    try:
        logger.info(f"Fetching static HTML from: {url}")
        response = requests.get(url, headers=HEADERS, timeout=30)
        response.raise_for_status()
        return response.text
    except requests.RequestException as e:
        logger.error(f"Failed to fetch {url}: {e}")
        raise

async def get_html_dynamic(url: str) -> str:
    """
    Fetches HTML content using a headless browser (Playwright).
    Necessary for the MLS site which may use JavaScript or require cookie consent.
    """
    logger.info(f"Launching Playwright for: {url}")
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(viewport={'width': 1920, 'height': 1080})
        page = await context.new_page()
        
        try:
            await page.goto(url, wait_until='domcontentloaded', timeout=60000)
            
            # Attempt to handle cookie consent popups if they exist
            try:
                consent_button = page.locator("button:has-text('Accept'), button:has-text('Consent')").first
                if await consent_button.is_visible():
                    await consent_button.click(timeout=5000)
                    logger.info("Clicked cookie consent button.")
            except Exception:
                logger.debug("No cookie consent button found or needed.")

            await page.wait_for_selector('main', timeout=60000)
            await page.wait_for_timeout(2000) 
            
            html_content = await page.content()
            return html_content
            
        except Exception as e:
            logger.error(f"Playwright error on {url}: {e}")
            raise
        finally:
            await browser.close()

## Parsing Logic Functions

In [0]:
def parse_packed_latlon(coords_str: str) -> tuple[Optional[float], Optional[float]]:
    if not coords_str: return None, None
    coords_str = str(coords_str).strip()
    match = PACKED_LATLON_RE.match(coords_str)
    if match:
        return float(match.group(1)), float(match.group(2))
    return None, None

def parse_stadium_coords(html: str) -> pd.DataFrame:
    try:
        tables = pd.read_html(StringIO(html))
        if not tables: return pd.DataFrame()
        df = tables[0].copy()
        df.columns = [str(c).strip().lower() for c in df.columns]
        name_col = next((c for c in ["location name", "location", "name"] if c in df.columns), df.columns[0])
        lat_col = "latitude" if "latitude" in df.columns else df.columns[1]
        lon_col = "longitude" if "longitude" in df.columns else None
        results = []
        for _, row in df.iterrows():
            stadium = str(row[name_col]).strip()
            lat_raw = str(row[lat_col]).strip()
            lat, lon = None, None
            if lon_col and pd.notna(row.get(lon_col)):
                try:
                    lat, lon = float(lat_raw), float(row[lon_col])
                except ValueError:
                    lat, lon = parse_packed_latlon(lat_raw)
            else:
                lat, lon = parse_packed_latlon(lat_raw)
            if lat is not None:
                results.append({"stadium": stadium, "lat": lat, "lon": lon})
        return pd.DataFrame(results).drop_duplicates(subset=["stadium"])
    except Exception as e:
        print(f"Error parsing coords: {e}")
        return pd.DataFrame()

def parse_world_cup_schedule(html: str) -> pd.DataFrame:
    soup = BeautifulSoup(html, "html.parser")
    main_content = soup.find("main") or soup
    text = main_content.get_text("\n")
    lines = [line.replace("\xa0", " ").strip() for line in text.splitlines() if line.strip()]
    
    match_rows = []
    line_index = 0
    
    current_city = None
    current_country = None
    current_stadium = None
    current_phase = "Group Stage"
    
    STADIUM_KEYWORDS = ["Stadium", "Field", "Estadio", "BC Place", "BMO Field", "Arena"]

    while line_index < len(lines):
        line = lines[line_index]
        
        is_date_line = line.startswith("June") or line.startswith("July")
        header_match = STADIUM_HDR_RE.match(line)
        is_third_place_line = "Third Place" in line or "Bronze" in line
        
        if header_match and not is_date_line and not is_third_place_line:
            stadium_part = header_match.group("stadium")
            if any(k in stadium_part for k in STADIUM_KEYWORDS):
                current_city = header_match.group("city").strip()
                current_stadium = stadium_part.strip()
                if "," in current_city: 
                    current_city = current_city.split(",")[0].strip()
                if line_index + 1 < len(lines) and lines[line_index+1] in COUNTRIES:
                    current_country = lines[line_index+1]
                line_index += 1
                continue

        if line in ["Group Stage", "Knockout Stage"]:
            current_phase = line
            line_index += 1
            continue
            
        date_match = MONTH_RE.match(line)
        if date_match and current_stadium:
            month, day, raw_details = date_match.groups()
            match_date = f"{month} {day}, 2026"
            raw_details = raw_details.strip() if raw_details else ""
            
            if not raw_details and (line_index + 1 < len(lines)):
                next_line = lines[line_index + 1].strip()
                is_next_header = STADIUM_HDR_RE.match(next_line) and any(k in next_line for k in STADIUM_KEYWORDS)
                is_next_third_place = "Third Place" in next_line or "Bronze" in next_line
                
                if (not is_next_header or is_next_third_place) and next_line not in COUNTRIES:
                    raw_details = next_line
                    line_index += 1 
            
            if raw_details:
                group_info = "Group Stage" if current_phase == "Group Stage" else current_phase
                teams_info = raw_details
                if " | " in raw_details:
                    parts = raw_details.split(" | ", 1)
                    group_info = parts[0].strip()
                    teams_info = parts[1].strip()
                elif "(" in raw_details and raw_details.endswith(")"):
                    parts = raw_details.split("(")
                    teams_info = parts[0].strip()
                    group_info = parts[1].replace(")", "").strip()

                match_rows.append({
                    "Date": match_date,
                    "City": current_city,
                    "Country": current_country,
                    "Stadium": current_stadium,
                    "Phase": current_phase,
                    "Group_Stage": group_info,
                    "Match_Teams": teams_info,
                    "Stadium_Key": STADIUM_SYNONYMS.get(current_stadium, current_stadium)
                })
            
        line_index += 1
        
    return pd.DataFrame(match_rows)

## Pipeline Execution

### Fetch Data

In [0]:
# Fetch World Cup 2026 Data
print("Fetching World Cup 2026 Schedule...")
loop = asyncio.get_event_loop()
raw_schedule_html = loop.run_until_complete(get_html_dynamic(WC_SCHEDULE_URL))

# Fetch LatLong Data
print("Fetching Coordinates...")
raw_latlong_html = get_html_static(LATLONG_URL)

print("Fetching complete.")
print(f"Schedule HTML Length: {len(raw_schedule_html)} chars")
print(f"LatLong HTML Length: {len(raw_latlong_html)} chars")

Fetching World Cup 2026 Schedule...
Fetching Coordinates...
Fetching complete.
Schedule HTML Length: 423979 chars
LatLong HTML Length: 37160 chars


### Parse, Merge & Save

In [0]:
# Parsing
print("Parsing World Cup matches...")
df_matches = parse_world_cup_schedule(raw_schedule_html)
print(f"Found {len(df_matches)} matches.")

print("Parsing coordinates...")
df_coords = parse_stadium_coords(raw_latlong_html)
print(f"Found {len(df_coords)} locations.")

# Merging
print("Merging Data...")
df_final = df_matches.merge(
    df_coords, 
    left_on="Stadium_Key", 
    right_on="stadium", 
    how="left"
)

# Clean & Rename
df_final = df_final.rename(columns={
    "Date": "date",
    "City": "city",
    "Country": "country",
    "Stadium": "stadium_name",
    "Phase": "phase",
    "Group_Stage": "group_or_stage",
    "Match_Teams": "match_teams",
    "lat": "latitude",
    "lon": "longitude"
})
final_cols = ["date", "city", "country", "stadium_name", "phase", "group_or_stage", "match_teams", "latitude", "longitude"]
df_final = df_final[final_cols]

# Save to Delta
print(f"Saving to Delta Table: {DELTA_TABLE_NAME}")
spark_df = spark.createDataFrame(df_final)
(spark_df.write
    .format("delta")
    .mode("overwrite")
    .partitionBy("country")
    .option("overwriteSchema", "true")
    .saveAsTable(DELTA_TABLE_NAME)
)

print(f"Success! {len(df_final)} matches saved.")
display(spark_df)

Parsing World Cup matches...
Found 104 matches.
Parsing coordinates...
Found 16 locations.
Merging Data...
Saving to Delta Table: world_cup_anastasia
Success! 104 matches saved.


date,city,country,stadium_name,phase,group_or_stage,match_teams,latitude,longitude
"June 15, 2026",Atlanta,United States,Mercedes-Benz Stadium,Group Stage,Spain vs. Cape Verde,Group H,33.755371,-84.401436
"June 18, 2026",Atlanta,United States,Mercedes-Benz Stadium,Group Stage,UEFA Playoff D Winner vs. South Africa,Group A,33.755371,-84.401436
"June 21, 2026",Atlanta,United States,Mercedes-Benz Stadium,Group Stage,Spain vs. Saudi Arabia,Group H,33.755371,-84.401436
"June 24, 2026",Atlanta,United States,Mercedes-Benz Stadium,Group Stage,Morocco vs. Haiti,Group C,33.755371,-84.401436
"June 27, 2026",Atlanta,United States,Mercedes-Benz Stadium,Group Stage,Intercontinental Playoff 1 Winner vs. Uzbekistan,Group K,33.755371,-84.401436
"July 1, 2026",Atlanta,United States,Mercedes-Benz Stadium,Knockout Stage,Round of 32,Group L Winner vs. Group E/H/I/J/K third place,33.755371,-84.401436
"July 7, 2026",Atlanta,United States,Mercedes-Benz Stadium,Knockout Stage,Round of 16,Winner Match 86 vs. Winner Match 88,33.755371,-84.401436
"July 15, 2026",Atlanta,United States,Mercedes-Benz Stadium,Knockout Stage,Semifinals,Winner Match 99 vs. Winner Match 100,33.755371,-84.401436
"June 13, 2026",Boston,United States,Gillette Stadium,Group Stage,Haiti vs. Scotland,Group C,42.09079,-71.264404
"June 16, 2026",Boston,United States,Gillette Stadium,Group Stage,Intercontinental Playoff 2 Winner vs. Norway,Group I,42.09079,-71.264404
